# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from dotenv import load_dotenv
load_dotenv("F:\Social AI\multiagent-framework\.venv\.env")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
# SERPER_API_KEY = in .env

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [16]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[ search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [17]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "Use the instruction folder to tailor your email."
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, directory_read_tool, file_read_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [18]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=True,
	memory=True
)

2024-12-01 13:09:51,788 - 2840 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


# Agent: Sales Representative
## Thought: I need to gather more information about DeepLearningAI, especially focusing on key decision-makers, recent business developments, potential needs, and case studies or projects. This will help in ensuring a comprehensive profile that can guide our engagement strategy effectively.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"DeepLearningAI company profile key personnel recent developments needs\"}"
## Tool Output: 


- Display the final result as Markdown.

In [21]:
from IPython.display import Markdown
Markdown(result.raw)

**Email Draft 1: Subject: Empowering AI Education Together**

Dear Andrew,

I hope this message finds you well! I wanted to extend my warm congratulations on the recent launch of the "Generative AI for Everyone" course. It's a remarkable initiative that truly embodies DeepLearningAI's mission to make AI education accessible to a global audience.

At CrewAI, we share your passion for transforming education through technology. Our customized AI solutions can significantly support your goals in curriculum development and learner data analytics. By integrating advanced AI tools into your platform, we can enhance the learning experience, enabling personalized pathways for your students.

Imagine leveraging our analytics capabilities to gain deeper insights into learner engagement, allowing you to continually evolve your offerings and meet the diverse needs of your audience. Together, we can empower a workforce ready to harness the full potential of AI.

I would love the opportunity to discuss this further and explore how we can collaborate to amplify the impact of your incredible work at DeepLearningAI.

Best regards,  
[Your Name]  
Lead Sales Representative, CrewAI  

---

**Email Draft 2: Subject: Enhancing Your Generative AI Impact**

Hi Andrew,

Congratulations on the successful launch of the “Generative AI for Everyone” course! It's exciting to see how DeepLearningAI is pushing the boundaries of AI education and accessibility.

At CrewAI, we're dedicated to innovation in educational technologies. Our bespoke AI solutions are designed to enhance curriculum development, streamline content delivery, and engage learners on a deeper level. As you expand your offerings, our expertise in learner data analytics can provide vital insights to tailor your courses even further.

We appreciate your vision of empowering a global workforce, and we believe that together, we can revolutionize the learning experience for countless individuals eager to explore the world of AI. I’d appreciate the chance to connect and discuss how CrewAI can support your ambitious goals.

Looking forward to hearing from you!

Kind regards,  
[Your Name]  
Lead Sales Representative, CrewAI  

---

**Email Draft 3: Subject: Collaborating for AI Education Excellence**

Dear Andrew,

I hope this note finds you in good spirits! I’m thrilled to see DeepLearningAI's new course on Generative AI come to fruition—what a fantastic way to democratize AI knowledge for everyone, irrespective of their technical background.

As you continue to innovate within the AI education space, I want to highlight how CrewAI's customized AI solutions could further augment your efforts. Our tools are perfectly tailored for enhancing educational technologies, making personalized learning experiences a reality for students globally.

By incorporating our advanced solutions, DeepLearningAI can not only optimize course offerings but also gain valuable insights through our learner analytics platform. Imagine the potential for your courses when every learner’s journey is enriched with data-driven personalization!

I’m eager to explore the potential of our collaboration. Please let me know a time that works for you to discuss this exciting opportunity.

Best wishes,  
[Your Name]  
Lead Sales Representative, CrewAI